In [1]:
import numpy as np
import pandas as pd

from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import cross_validate

In [2]:
data = pd.read_csv('labeled.csv')

In [3]:
data

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0
...,...,...
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1.0
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1.0


Задание 2

In [4]:
text_3 = data.iloc[3]['comment']
text_12666 = data.iloc[12666]['comment']
print(text_3)
print(text_12666)

Страницу обнови, дебил. Это тоже не оскорбление, а доказанный факт - не-дебил про себя во множественном числе писать не будет. Или мы в тебя верим - это ты и твои воображаемые друзья?

Это не параноик, это дебил



In [5]:
#векторизуем тексты
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['comment'])

In [6]:
print(X[3])

  (0, 67953)	2
  (0, 31914)	3
  (0, 57846)	1
  (0, 34207)	1
  (0, 13512)	2
  (0, 59921)	1
  (0, 36274)	1
  (0, 14953)	1
  (0, 63446)	1
  (0, 46725)	1
  (0, 53334)	1
  (0, 8621)	1
  (0, 29143)	1
  (0, 66163)	1
  (0, 40271)	1
  (0, 6474)	1
  (0, 20563)	1
  (0, 30068)	1
  (0, 59246)	1
  (0, 7492)	1
  (0, 61104)	1
  (0, 59196)	1
  (0, 9083)	1
  (0, 15858)	1


In [7]:
print(X[12666])

  (0, 67953)	2
  (0, 31914)	1
  (0, 13512)	1
  (0, 38372)	1


Надо было наверное удалить знаки препинания, потому что сейча «дебил» и «дебил.» — разные слова

In [8]:
#найдём косинусную близость между ними
cosine_similarity(X[3], X[12666])

array([[0.55182541]])

Найдём косинусную близость с текстом 43 для каждого текста

In [9]:
data.iloc[43]['comment']

'Люди зажрались и охуели если по мнению этих игроков андромеда лучше Антема. Хотя че там с багаутом76 сравнивают вон... Вот оно че оказывается, игроки просто охуели, ну ок. Вот долбанные пидерасы, не хотят покупать такую отличную игру, а еще смеют ругать такой божественный геймплей. Баги, хуевый геймплей поправят, а контент, ну его запилят, через год другой, причем бесплатно! Ага. Стоит лишь потерпеть!\n'

In [10]:
data = pd.concat([data, pd.DataFrame(X, columns=['vector'])], axis=1)

In [11]:
data

,comment,toxic,vector
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0,"(0, 7474)\t1\n (0, 59887)\t1\n (0, 17206)\..."
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0,"(0, 65040)\t1\n (0, 67953)\t1\n (0, 36839)..."
2,Собаке - собачья смерть\n,1.0,"(0, 55308)\t1\n (0, 55320)\t1\n (0, 54991)\t1"
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0,"(0, 67953)\t2\n (0, 31914)\t3\n (0, 57846)..."
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0,"(0, 66317)\t2\n (0, 31914)\t1\n (0, 59246)..."
...,...,...,...
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1.0,"(0, 9314)\t1\n (0, 54719)\t1\n (0, 53072)\..."
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0,"(0, 66317)\t1\n (0, 31914)\t1\n (0, 20563)..."
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0,"(0, 66317)\t2\n (0, 67953)\t1\n (0, 65279)..."
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1.0,"(0, 66317)\t1\n (0, 67953)\t2\n (0, 16551)..."


In [12]:
data['sim_to_43'] = data['vector'].apply(lambda x: cosine_similarity(x, X[43])[0])

In [13]:
data

,comment,toxic,vector,sim_to_43
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0,"(0, 5534)\t1\n (0, 7474)\t1\n (0, 13525)\t...",[0.0]
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0,"(0, 6684)\t2\n (0, 6755)\t1\n (0, 9067)\t1...",[0.10114434748483472]
2,Собаке - собачья смерть\n,1.0,"(0, 54991)\t1\n (0, 55308)\t1\n (0, 55320)\t1",[0.0]
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0,"(0, 6474)\t1\n (0, 7492)\t1\n (0, 8621)\t1...",[0.05901671065234753]
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0,"(0, 3563)\t1\n (0, 7082)\t1\n (0, 15924)\t...",[0.028583097523751475]
...,...,...,...,...
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1.0,"(0, 9074)\t1\n (0, 9314)\t1\n (0, 13810)\t...",[0.07312724241271307]
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0,"(0, 11974)\t1\n (0, 13838)\t1\n (0, 20200)...",[0.028583097523751475]
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0,"(0, 4063)\t1\n (0, 6687)\t1\n (0, 6793)\t1...",[0.09469425238482128]
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1.0,"(0, 4928)\t1\n (0, 12227)\t1\n (0, 15425)\...",[0.0199363055707225]


In [14]:
data.iloc[43]['sim_to_43']

array([1.])

In [15]:
#отсортируем по убыванию
sorted_data = data.sort_values(by=['sim_to_43'], ascending=False)
sorted_data

,comment,toxic,vector,sim_to_43
43,Люди зажрались и охуели если по мнению этих иг...,1.0,"(0, 2909)\t1\n (0, 3653)\t1\n (0, 3742)\t1...",[0.9999999999999989]
14325,"Ну оль, ну вот че тебе надо? Иди в по сри, а э...",1.0,"(0, 9314)\t1\n (0, 15545)\t1\n (0, 20158)\...",[0.2647058823529412]
13132,"Да, тяжелая штука. Важно помнить - ты в этой ж...",1.0,"(0, 6880)\t1\n (0, 9314)\t3\n (0, 11079)\t...",[0.2296213588413651]
7991,Кастрировать не мучать . Ну ну.\n,1.0,"(0, 22363)\t1\n (0, 30044)\t1\n (0, 31914)...",[0.2291746242570528]
3099,"Ну вот, отдел маркетинга сработал )\n",0.0,"(0, 9314)\t1\n (0, 27735)\t1\n (0, 33678)\...",[0.21693045781865616]
...,...,...,...,...
6788,Из-за яек подбитых сошел?\n,1.0,"(0, 17206)\t1\n (0, 20200)\t1\n (0, 41249)...",[0.0]
6782,в кинотеатре был последний раз на премьере тер...,0.0,"(0, 5969)\t1\n (0, 6750)\t1\n (0, 9515)\t1...",[0.0]
6779,ищу коммент про нарушение тб\n,1.0,"(0, 21623)\t1\n (0, 23567)\t1\n (0, 31314)...",[0.0]
6777,"Такое первые пару раз, потом нормально все, ра...",0.0,"(0, 9529)\t1\n (0, 13314)\t1\n (0, 25134)\...",[0.0]


In [16]:
#и посмотрим на первые 3 без самого текста 43
sorted_data.iloc[1:4]

,comment,toxic,vector,sim_to_43
14325,"Ну оль, ну вот че тебе надо? Иди в по сри, а э...",1.0,"(0, 9314)\t1\n (0, 15545)\t1\n (0, 20158)\...",[0.2647058823529412]
13132,"Да, тяжелая штука. Важно помнить - ты в этой ж...",1.0,"(0, 6880)\t1\n (0, 9314)\t3\n (0, 11079)\t...",[0.2296213588413651]
7991,Кастрировать не мучать . Ну ну.\n,1.0,"(0, 22363)\t1\n (0, 30044)\t1\n (0, 31914)...",[0.2291746242570528]


Задание 3

In [17]:
#загружаем текст с двача
X_dvach = open('2ch_corpus.txt', encoding='utf8').read()[:700000]
X_dvach = sent_tokenize(X_dvach)

In [18]:
X_dvach[57]

'Вставки из глав скорее особенность подачи и монтажа, но никак не пример четвертой стены.'

In [19]:
#хочется завернуть векторизатор и классификатор в один класс чтобы сделать честную кросс-валидацию и векторизовать только тренировочную выборку
class Vectorizer_and_Classifier:
    def __init__(self, vectorizer=None, classifier=None):
        self.vectorizer = vectorizer
        self.classifier = classifier
    def get_params(self, deep=False):
        return {'vectorizer' : self.vectorizer, 'classifier' : self.classifier}
    def fit(self, X, y):
        #векторизуем
        X = self.vectorizer.fit_transform(X)
        #обучаем классификатор
        self.classifier.fit(X, y)
    def predict(self, X):
        X = self.vectorizer.transform(X)
        return self.classifier.predict(X)
    def predict_proba(self, X):
        X = self.vectorizer.transform(X)
        return self.classifier.predict_proba(X)

In [20]:
def cross_validation(vectorizer, clsassifier):
    vc = Vectorizer_and_Classifier(vectorizer, clsassifier)
    scores = cross_validate(vc, data['comment'], data['toxic'], cv=10, scoring='f1')
    return scores['test_score']

In [21]:
def top(n, data, y):
    y = y[:, 1]
    sorted_ind = np.argsort(-y)
    for i in range(n):
        index = sorted_ind[i]
        print(y[index])
        print(data[index])

In [22]:
def predict_dvach(vectorizer, clsassifier):
    vc = Vectorizer_and_Classifier(vectorizer, clsassifier)
    #обучаем на всём корпусе
    vc.fit(data['comment'], data['toxic'])
    # предсказываем
    y_pred = vc.predict_proba(X_dvach)
    print(y_pred)
    top(10, X_dvach, y_pred)

In [23]:
from nltk.corpus import stopwords

In [24]:
stop_words = stopwords.words("russian")

Для первого классификатора

In [25]:
vectorizator = CountVectorizer(stop_words=stop_words, max_df=400, min_df=0, lowercase=True, ngram_range=(1,3))
classifier = LogisticRegression(C=0.35, class_weight='balanced', solver='lbfgs', max_iter=700)

In [26]:
# делаем кросс-валидацию чтобы подобрать параметры
scores = cross_validation(vectorizator, classifier)
print(scores)
print(np.mean(scores))

[0.66666667 0.83615222 0.75862069 0.7631242  0.83796296 0.6746988
 0.72798434 0.65217391 0.66666667 0.8612153 ]
0.7445265762041395


In [27]:
# предсказываем токсичность и выводим топ-10
predict_dvach(vectorizer, classifier)

[[0.29468069 0.70531931]
 [0.6609832  0.3390168 ]
 [0.41677127 0.58322873]
 ...
 [0.42233627 0.57766373]
 [0.57308523 0.42691477]
 [0.62921423 0.37078577]]
0.9999980266920903
Ты просто сам себя в этом убедил, а мог бороться за свои права, я бы и мусорам и друзьямзвонил всех бы подтянул, у нас как бы демократия блять, хотя порашные треды и кукарекают о полицейском беспределе, мусора блять первые кто тебе помогут, ты можешь просто сославшись на закон о правах пациентов, уверен висит на входе в вашу дурку, сказать что тебе не дают ни с главврачом побеседовать, ни заявление написать, все, ты свободен сука
 Ну не знаю, проблемы у меня начались после смерти отца.
0.9999969476157159
самопочин
 Сука помогите пиздец глаз красный чешется я его чешу уже 2 часа ща сдохну нахуй пиздец как он чешется бляяяяядь
 Сука помогите пиздец глаз красный чешется я его чешу уже 2 часа ща сдохну нахуй пиздец как он чешется бляяяяядь
 Под струю воды сунь, ссунь.
0.9999935092582352
В итоге дибил это то, кто такие

Казалось бы почти все тексты оказались токчисными, но вот второй текст показывает, что мы получили скорее тексты, насыщенные «токсичными» словами

Для второго классификатора

In [28]:
vectorizator = TfidfVectorizer(stop_words=stop_words, max_df=0.9, min_df=0, lowercase=True, ngram_range=(1,2))
classifier = KNeighborsClassifier(n_neighbors=5, metric='cosine', weights='uniform')

In [29]:
# делаем кросс-валидацию чтобы подобрать параметры
scores = cross_validation(vectorizator, classifier)
print(scores)
print(np.mean(scores))

[0.65275708 0.76363636 0.67540029 0.66666667 0.76480541 0.54273504
 0.55291577 0.52025586 0.5738758  0.7649919 ]
0.6478040187245333


Не очень понял почему на паре скор получался настолько больше, видимо дело в разделении на выборки 

In [30]:
# предсказываем токсичность и выводим топ-10
predict_dvach(vectorizer, classifier)

[[0.4 0.6]
 [0.8 0.2]
 [0.8 0.2]
 ...
 [0.4 0.6]
 [0.8 0.2]
 [0.8 0.2]]
1.0
Аноны.
1.0
рыжие волосы и говноглазка.
1.0
У него пиздец долгов было и он одолжил у чела и договорился, что он напишет роман за год, иначе все права будущих произведений переходят от достоевского к тому знакомому.
1.0
Пойду в Си тред
 В де хочет сьебать.
1.0
У тебя в голове идеализированный образ.
1.0
Ебланы, блять.
1.0
Или когда ты табуретку делаешь, ты ведь тоже творишь.
1.0
Ты в шаге от того, чтобы назвать бред шизофреника - творчеством.
1.0
Ты там дрочить собрался?
1.0
Кому это нужно?Можешь писать как угодно, тут вопрос только в том, хочешь ли ты, чтобы тебя читали или нет.


Тут у многих текстов получилась вероятность 1 (все соседи токсичные) и топ 10 в целом хуже